In [1]:
import pandas as pd
import numpy as np
# from scipy.stats import pearsonr, median_abs_deviation
# from sklearn import metrics
import sys
import os
from statsmodels.stats.multitest import multipletests
from scipy.stats import combine_pvalues, norm, mannwhitneyu, spearmanr, pearsonr
from read_clustering.variant_call import VariantCalls
import matplotlib.pyplot as plt
from scipy.stats import shapiro, combine_pvalues
import seaborn as sns
import re
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as mpatches
from scipy.stats import combine_pvalues, norm, mannwhitneyu
from scipy.stats import chi2_contingency

/Users/andrewbailey/anaconda3/envs/read_clustering/lib/python3.7/site-packages/umap/__init__.py:9: UserWarning: Tensorflow not installed; ParametricUMAP will be unavailable
  warn("Tensorflow not installed; ParametricUMAP will be unavailable")
/Users/andrewbailey/anaconda3/envs/read_clustering/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
# final ivt vs native
CBF5GAL = "modification_calls/20190610_R941_CBF5GAL.csv"
CBF5GLU = "modification_calls/20190611_R941_CBF5GLU.csv"
NOP58GAL = "modification_calls/20190617_R941_NOP58GAL.csv"
NOP58GLU = "modification_calls/20190619_R941_NOP58GLU.csv"
IVT = "modification_calls/20191014_R941_IVT.csv"
mutant448 = "modification_calls/20210405_R941_mutant448.csv"
WT_YPD = "modification_calls/20210410_R941_WT_YPD.csv"
WT_KOAC = "modification_calls/20210412_R941_WT_KOAC.csv"
mutant452 = "modification_calls/20210413_R941_mutant452.csv"
WT_CY = "modification_calls/20210414_R941_WT_CY.csv"
mutant451 = "modification_calls/20210415_R941_mutant451.csv"
WT_PB = "modification_calls/20210520_R941_WT_PB.csv"
WT_stationary = "modification_calls/20210521_R941_WT_stationary.csv"
Dbp3d_YPD = "modification_calls/20210527_R941_Dbp3d_YPD.csv"
trl1d_YPD = "modification_calls/20210528_R941_trl1d_YPD.csv"
WT_rapa5 = "modification_calls/20210531_R941_WT_rapa5.csv"
Mix = "modification_calls/20210618_R941_Mix.csv"
prp43 = "modification_calls/20210619_R941_prp43.csv"
MIX_2 = "modification_calls/20210716_R941_MIX.csv"
YS602_18_YPD = "modification_calls/20210717_R941_YS602_18_YPD.csv"
prp16_18deg_YPD = "modification_calls/20210719_R941_prp16_18deg_YPD.csv"
spp382_YPD = "modification_calls/20210720_R941_spp382_YPD.csv"
YS602_18_YPD2 = "modification_calls/20210722_R941_YS602_18_YPD2.csv"
dbr1d_YPD = "modification_calls/20210729_R941_dbr1d_YPD.csv"
sqs1d_YPD = "modification_calls/20210730_R941_sqs1d_YPD.csv"
snR4d_YPD = "modification_calls/20210823_R941_snR4d_YPD.csv"
snR45d_YPD = "modification_calls/20210824_R941_snR45d_YPD.csv"
mutant448_2 = "modification_calls/20210825_R941_mutant448.csv"
snR4d_alpha_YPD = "modification_calls/20210902_R941_snR4d_alpha_YPD.csv"
WT_YPD2 = "modification_calls/20210930_R941_YS602_WT_repeat2.csv"
WT_YPD3 = "modification_calls/20211001_R941_YS602_WT_repeat3.csv"
prx1 = "modification_calls/20211006_R941_pxr1d_YPD.csv"
YS602_18_YPD3 = "modification_calls/20211007_R941_YS602_18_YPD3.csv"
prp43_2 = "modification_calls/20211008_R941_prp43_18deg_YPD_repeat2.csv"
Dbp3d_YPD2 = "modification_calls/20211011_R941_dbp3d_YPD_repeat2.csv"
prx1_2 = "modification_calls/20211021_pxr1d_YPD_repeat2.csv"


In [3]:
correlation_df = pd.read_csv("figures/summary_data/all_experiment_correlations_corrected_pvalues.csv", 
                                   index_col=0)
chi2_pvalues_df = pd.read_csv("figures/summary_data/all_experiment_percent_modification_corrected_pvalues_chi2.csv", 
                                   index_col=0)


In [4]:
def get_max_correlation_diff_pvalues(labels, contig, compare_labels=["WT_YPD", "WT_YPD2", "WT_YPD3"], diff=True):
    exp_pvalues = []
    exp_comparison_pvalues = []
    exp_diff_values = []
    exp_corr = []
    
    if not isinstance(labels, list):
        labels = [labels]        
    
    for label in labels:
        for compare in compare_labels:
            exp_comparison_pvalues.append(correlation_df[f"{label}_corrected_{compare}_comparison_pvalue"][correlation_df["contig"] == contig])
            exp_pvalues.append(correlation_df[f"{label}_corrected_pvalue"][correlation_df["contig"] == contig])
            exp_diff_values.append((correlation_df[f"{label}_corr"] - correlation_df[f"{compare}_corr"])[correlation_df["contig"] == contig])
            exp_corr.append((correlation_df[f"{label}_corr"])[correlation_df["contig"] == contig])

    all_diffs_df = pd.DataFrame(exp_diff_values)
    all_pvalues_df = pd.DataFrame(exp_pvalues, index=list(range(len(exp_pvalues))))
    all_comparison_pvalues_df = pd.DataFrame(exp_comparison_pvalues, index=list(range(len(exp_pvalues))))
    all_exp_corr_df = pd.DataFrame(exp_corr, index=list(range(len(exp_pvalues))))
    
    per_position_index = np.abs(all_comparison_pvalues_df).idxmax()
    exp_diff_values = pd.Series(all_diffs_df.lookup(per_position_index.values, per_position_index.index), index=per_position_index.index)
    exp_comparison_pvalues = pd.Series(all_comparison_pvalues_df.lookup(per_position_index.values, per_position_index.index), index=per_position_index.index)


    per_position_index = np.abs(all_pvalues_df).idxmax()
    exp_pvalues = pd.Series(all_pvalues_df.lookup(per_position_index.values, per_position_index.index), index=per_position_index.index)
    exp_corr_values = pd.Series(all_exp_corr_df.lookup(per_position_index.values, per_position_index.index), index=per_position_index.index)


    exp_diff = correlation_df[["ref_index1", "ref_index2"]][correlation_df["contig"] == contig]
    if diff:
        exp_diff["corr_diff"] = exp_diff_values
    else:
        exp_diff["corr_diff"] = exp_corr_values

    exp_diff = exp_diff.pivot(index="ref_index2", columns='ref_index1', values='corr_diff')


    mapper = list(set(correlation_df["ref_index1"][correlation_df["contig"] == contig]) | set(correlation_df["ref_index2"][correlation_df["contig"] == contig]))
    mapper.sort()
    return exp_diff, mapper, exp_pvalues, exp_comparison_pvalues


# P-value calculations

## Depletions

In [5]:
pseduo_u_pos = [775, 959, 965, 985, 989, 1003, 1041, 1051, 1055, 1109, 1123,
                     2128, 2132, 2190, 2257, 2259, 2263, 2265, 2313, 2339, 2348, 2350,
                     2415, 2734, 2825, 2864, 2879, 2922, 2943, 2974, 105, 119, 210, 301, 465, 631, 758, 765,
                     998, 1180, 1186,
                     1289, 1414]
twoprimeo_pos = [648, 649, 662, 804, 806, 816, 866, 875, 897, 907, 1132,
                      1436, 1448, 1449, 1887, 2196, 2219, 2255, 2279, 2280, 2287, 2336,
                      2346, 2416, 2420, 2618, 2639, 2723, 2728, 2790, 2792, 2814, 2920,
                      2921, 2945, 2947, 2958, 27, 99, 413, 419, 435, 540, 561, 577, 618, 795, 973,
                      1006, 1125, 1268, 1270, 1427, 1571, 1638]
# https://keisan.casio.com/exec/system/1180573196

In [6]:
exp_diff, mapper, exp_pvalues, exp_comparison_pvalues = get_max_correlation_diff_pvalues("NOP58GLU", "RDN18-1")   
exp_diff1, mapper1, exp_pvalues1, exp_comparison_pvalues1 = get_max_correlation_diff_pvalues("NOP58GLU", "RDN25-1")   
exp_pvalues = pd.concat([exp_pvalues1, exp_pvalues])
exp_comparison_pvalues = pd.concat([exp_comparison_pvalues1, exp_comparison_pvalues])
twoprimeo_pvalues = []
other_pvalues = []
for index in range(len(correlation_df)):
    data = correlation_df.iloc[index]
    ref_index1 = data["ref_index1"]-1
    ref_index2 = data["ref_index2"]-1
    if ref_index1 in twoprimeo_pos and ref_index2 in twoprimeo_pos:
        twoprimeo_pvalues.append(np.max([exp_comparison_pvalues.iloc[index], exp_pvalues.iloc[index]]))
#         twoprimeo_pvalues.append()
    else:
        other_pvalues.append(np.max([exp_comparison_pvalues.iloc[index], exp_pvalues.iloc[index] ]))

statistic, pvalue = combine_pvalues(twoprimeo_pvalues)
print(statistic, pvalue, len(twoprimeo_pvalues))
statistic, pvalue = combine_pvalues(other_pvalues)
print(statistic, pvalue, len(other_pvalues))


13231.815112545999 0.0 819
2357.141408426775 1.0 2475


In [7]:
exp_diff, mapper, exp_pvalues, exp_comparison_pvalues = get_max_correlation_diff_pvalues("CBF5GLU", "RDN18-1")   
exp_diff1, mapper1, exp_pvalues1, exp_comparison_pvalues1 = get_max_correlation_diff_pvalues("CBF5GLU", "RDN25-1")   
exp_pvalues = pd.concat([exp_pvalues1, exp_pvalues])
exp_comparison_pvalues = pd.concat([exp_comparison_pvalues1, exp_comparison_pvalues])

pseudou_pvalues = []
other_pvalues = []
for index in range(len(correlation_df)):
    data = correlation_df.iloc[index]
    ref_index1 = data["ref_index1"]-1
    ref_index2 = data["ref_index2"]-1
    if ref_index1 in pseduo_u_pos and ref_index2 in pseduo_u_pos:
        pseudou_pvalues.append(np.max([exp_comparison_pvalues.iloc[index], exp_pvalues.iloc[index] ]))
    else:
        other_pvalues.append(np.max([exp_comparison_pvalues.iloc[index], exp_pvalues.iloc[index] ]))

statistic, pvalue = combine_pvalues(pseudou_pvalues)
print(statistic, pvalue, len(pseudou_pvalues))
statistic, pvalue = combine_pvalues(other_pvalues)
print(statistic, pvalue, len(other_pvalues))

2127.1706837993975 3.4784703911390786e-79 513
1661.0833883088205 1.0 2781


In [8]:
exp_diff, mapper, exp_pvalues, exp_comparison_pvalues = get_max_correlation_diff_pvalues("NOP58GLU", "RDN18-1")   
exp_diff1, mapper1, exp_pvalues1, exp_comparison_pvalues1 = get_max_correlation_diff_pvalues("NOP58GLU", "RDN25-1")   
exp_pvalues = pd.concat([exp_pvalues1, exp_pvalues])
exp_comparison_pvalues = pd.concat([exp_comparison_pvalues1, exp_comparison_pvalues])

twoprimeo_pvalues = []
other_pvalues = []
for index in range(len(correlation_df)):
    data = correlation_df.iloc[index]
    ref_index1 = data["ref_index1"]-1
    ref_index2 = data["ref_index2"]-1
    if ref_index1 in [1772, 1279] and ref_index2 in twoprimeo_pos:
        twoprimeo_pvalues.append(np.max([exp_comparison_pvalues.iloc[index], exp_pvalues.iloc[index]]))

statistic, pvalue = combine_pvalues(twoprimeo_pvalues)
print(statistic, pvalue)


32.80980606685657 1.140737291240036e-05


In [10]:
exp_diff, mapper, exp_pvalues, exp_comparison_pvalues = get_max_correlation_diff_pvalues("CBF5GLU", "RDN18-1")   
exp_diff1, mapper1, exp_pvalues1, exp_comparison_pvalues1 = get_max_correlation_diff_pvalues("CBF5GLU", "RDN25-1")   
exp_pvalues = pd.concat([exp_pvalues1, exp_pvalues])
exp_comparison_pvalues = pd.concat([exp_comparison_pvalues1, exp_comparison_pvalues])

twoprimeo_pvalues = []
other_pvalues = []
for index in range(len(correlation_df)):
    data = correlation_df.iloc[index]
    ref_index1 = data["ref_index1"]-1
    ref_index2 = data["ref_index2"]-1
    if ref_index1 in [1190] and ref_index2 in pseduo_u_pos:
        twoprimeo_pvalues.append(np.max([exp_comparison_pvalues.iloc[index], exp_pvalues.iloc[index]]))

statistic, pvalue = combine_pvalues(twoprimeo_pvalues)
print(statistic, pvalue)


30.22815485928323 4.397896651071692e-06


# Mixture and snoRNA KO's

In [95]:
exp_diff, mapper, exp_pvalues, exp_comparison_pvalues = get_max_correlation_diff_pvalues(["MIX_2", "Mix"], "RDN18-1")   
exp_diff1, mapper1, exp_pvalues1, exp_comparison_pvalues1 = get_max_correlation_diff_pvalues(["MIX_2", "Mix"], "RDN25-1")   
exp_pvalues = pd.concat([exp_pvalues1, exp_pvalues])
exp_comparison_pvalues = pd.concat([exp_comparison_pvalues1, exp_comparison_pvalues])
pos = [436, 759, 1290, 1415]

mix_pvalues = []
other_pvalues = []
for index in range(len(correlation_df)):
    data = correlation_df.iloc[index]
    ref_index1 = data["ref_index1"]
    ref_index2 = data["ref_index2"]
    if ref_index1 in pos and ref_index2 in pos:
        print(ref_index1, ref_index2)
        mix_pvalues.append(np.max([exp_comparison_pvalues.iloc[index], exp_pvalues.iloc[index]]))
    else:
        other_pvalues.append(np.max([exp_comparison_pvalues.iloc[index], exp_pvalues.iloc[index]]))

statistic, pvalue = combine_pvalues(mix_pvalues[:-1])
print(statistic, pvalue, len(mix_pvalues))
print(mix_pvalues)

436 759
436 1290
436 1415
759 1290
759 1415
1290 1415
42.694166838608496 5.6397700883854446e-06 6
[0.000334102469098484, 0.007653054862306545, 0.6346749046490494, 0.01566422448965682, 0.02108165633928185, 5.802674429543519e-14]


In [96]:
MIX = chi2_pvalues_df["MIX_2_mean"][[436, 759, 1290, 1415]]
WT = chi2_pvalues_df["WT_YPD_mean"][[436, 759, 1290, 1415]]
WT2 = chi2_pvalues_df["WT_YPD2_mean"][[436, 759, 1290, 1415]]
WT3 = chi2_pvalues_df["WT_YPD3_mean"][[436, 759, 1290, 1415]]

In [63]:
pd.DataFrame([(MIX - WT), (MIX - WT2), (MIX - WT3)])

reference_index,436,759,1290,1415
0,-0.163803,-0.228544,-0.185225,-0.141084
1,-0.176463,-0.230526,-0.202300,-0.134405
2,-0.171986,-0.243004,-0.199668,-0.161221


## Wild Type

In [97]:
exp_diff, mapper, exp_pvalues, exp_comparison_pvalues = get_max_correlation_diff_pvalues(["WT_YPD", "WT_YPD2", "WT_YPD3"], "RDN18-1", compare_labels=["IVT"])   
exp_diff1, mapper1, exp_pvalues1, exp_comparison_pvalues1 = get_max_correlation_diff_pvalues(["WT_YPD", "WT_YPD2", "WT_YPD3"], "RDN25-1", compare_labels=["IVT"])     
exp_pvalues = pd.concat([exp_pvalues1, exp_pvalues])
exp_comparison_pvalues = pd.concat([exp_comparison_pvalues1, exp_comparison_pvalues])


In [99]:
pos = [632, 766, 100 ,436, 1773, 1639]

for index in range(len(correlation_df)):
    data = correlation_df.iloc[index]
    ref_index1 = data["ref_index1"]
    ref_index2 = data["ref_index2"]
    if ref_index1 in pos and ref_index2 in pos:
        print(ref_index1, ref_index2, exp_comparison_pvalues.iloc[index], np.max([exp_comparison_pvalues.iloc[index], exp_pvalues.iloc[index]]))


100 436 0.005584536530788167 0.005584536530788167
100 632 0.9969042473955798 0.9969042473955798
100 766 0.4638187602088135 0.8676112083269425
100 1639 0.6580502215314652 0.8033820585540702
100 1773 0.6785647912251958 0.6785647912251958
436 632 0.2145592258329473 0.9705677725542508
436 766 0.6217280675751813 0.8151492175964206
436 1639 0.5562825857971044 0.8840708649640368
436 1773 0.06859652039098145 0.9917253261482758
632 766 0.003079197888448077 0.003079197888448077
632 1639 0.914400433800851 0.914400433800851
632 1773 0.3949229699935684 0.8573184868270026
766 1639 0.29634252489878593 0.5639528296160582
766 1773 0.9800256155107528 0.9800256155107528
1639 1773 0.00019617657556041245 0.0015845092513138285


In [104]:
pos = [1437, 1449, 1450]
data1 = []
for index in range(len(correlation_df)):
    data = correlation_df.iloc[index]
    ref_index1 = data["ref_index1"]
    ref_index2 = data["ref_index2"]
    if ref_index1 in pos and ref_index2 in pos:
        print(ref_index1, ref_index2, exp_comparison_pvalues.iloc[index], np.max([exp_comparison_pvalues.iloc[index], exp_pvalues.iloc[index]]))
        data1.append(np.max([exp_comparison_pvalues.iloc[index], exp_pvalues.iloc[index]]))
combine_pvalues(data1)

1437 1449 1.3237018945972188e-14 1.3237018945972188e-14
1437 1450 1.952964055443689e-11 1.952964055443689e-11
1449 1450 5.08165711416416e-37 5.08165711416416e-37


(280.3697156593953, 1.3093562668412848e-57)

## RNA helicases

In [19]:
exp_diff, mapper, exp_pvalues, exp_comparison_pvalues = get_max_correlation_diff_pvalues(["Dbp3d_YPD", "Dbp3d_YPD2"], "RDN18-1")   
exp_diff1, mapper1, exp_pvalues1, exp_comparison_pvalues1 = get_max_correlation_diff_pvalues(["Dbp3d_YPD", "Dbp3d_YPD2"], "RDN25-1")     
exp_pvalues = pd.concat([exp_pvalues1, exp_pvalues])
exp_comparison_pvalues = pd.concat([exp_comparison_pvalues1, exp_comparison_pvalues])
pos = [1437, 1449, 1450]
data1 = []
for index in range(len(correlation_df)):
    data = correlation_df.iloc[index]
    ref_index1 = data["ref_index1"]
    ref_index2 = data["ref_index2"]
    if ref_index1 in pos and ref_index2 in pos:
        print(ref_index1, ref_index2, exp_comparison_pvalues.iloc[index], np.max([exp_comparison_pvalues.iloc[index], exp_pvalues.iloc[index]]))
        data1.append(np.max([exp_comparison_pvalues.iloc[index], exp_pvalues.iloc[index]]))
print(combine_pvalues(data1))

pos = [817, 908, 2619, 2724]
for index in range(len(correlation_df)):
    data = correlation_df.iloc[index]
    ref_index1 = data["ref_index1"]
    ref_index2 = data["ref_index2"]
    if ref_index1 in pos and ref_index2 in pos:
        print(ref_index1, ref_index2, exp_comparison_pvalues.iloc[index], np.max([exp_comparison_pvalues.iloc[index], exp_pvalues.iloc[index]]))


1437 1449 3.8376704791242086e-45 3.8376704791242086e-45
1437 1450 1.0551679114364113e-44 1.0551679114364113e-44
1449 1450 2.2559696558147545e-12 2.2559696558147545e-12
(460.69789812162503, 2.444756959894857e-96)
817 908 2.6203220266931644e-32 2.6203220266931644e-32
817 2619 0.8509803381528454 0.8621788408430999
817 2724 0.7617278208422629 0.7617278208422629
908 2619 0.9923468576343126 0.9923468576343126
908 2724 0.9578635822295868 0.9578635822295868
2619 2724 1.3331965186791088e-20 1.3331965186791088e-20


In [20]:
exp_diff, mapper, exp_pvalues, exp_comparison_pvalues = get_max_correlation_diff_pvalues(["prp43", "prp43_2"], "RDN18-1")   
exp_diff1, mapper1, exp_pvalues1, exp_comparison_pvalues1 = get_max_correlation_diff_pvalues(["prp43", "prp43_2"], "RDN25-1")     
exp_pvalues = pd.concat([exp_pvalues1, exp_pvalues])
exp_comparison_pvalues = pd.concat([exp_comparison_pvalues1, exp_comparison_pvalues])
pos = [1437, 1449, 1450]
data1 = []
for index in range(len(correlation_df)):
    data = correlation_df.iloc[index]
    ref_index1 = data["ref_index1"]
    ref_index2 = data["ref_index2"]
    if ref_index1 in pos and ref_index2 in pos:
        print(ref_index1, ref_index2, exp_comparison_pvalues.iloc[index], np.max([exp_comparison_pvalues.iloc[index], exp_pvalues.iloc[index]]))
        data1.append(np.max([exp_comparison_pvalues.iloc[index], exp_pvalues.iloc[index]]))
print(combine_pvalues(data1))

pos = [817, 908, 2619, 2724]
for index in range(len(correlation_df)):
    data = correlation_df.iloc[index]
    ref_index1 = data["ref_index1"]
    ref_index2 = data["ref_index2"]
    if ref_index1 in pos and ref_index2 in pos:
        print(ref_index1, ref_index2, exp_comparison_pvalues.iloc[index], np.max([exp_comparison_pvalues.iloc[index], exp_pvalues.iloc[index]]))


1437 1449 9.97697184007468e-10 9.97697184007468e-10
1437 1450 0.0010303788281890265 0.0010303788281890265
1449 1450 0.9971135973662546 0.9971135973662546
(55.21258126965293, 4.1991733158394006e-10)
817 908 0.8374989219832035 0.8374989219832035
817 2619 0.9846930320842404 0.9846930320842404
817 2724 0.9822632465205277 0.9822632465205277
908 2619 0.9784569496047942 0.9784569496047942
908 2724 0.9494154838115187 0.9494154838115187
2619 2724 0.9922302288171346 0.9922302288171346


In [21]:
exp_diff, mapper, exp_pvalues, exp_comparison_pvalues = get_max_correlation_diff_pvalues(["prx1", "prx1_2"], "RDN18-1")   
exp_diff1, mapper1, exp_pvalues1, exp_comparison_pvalues1 = get_max_correlation_diff_pvalues(["prx1", "prx1_2"], "RDN25-1")     
exp_pvalues = pd.concat([exp_pvalues1, exp_pvalues])
exp_comparison_pvalues = pd.concat([exp_comparison_pvalues1, exp_comparison_pvalues])
pos = [1437, 1449, 1450]
data1 = []
for index in range(len(correlation_df)):
    data = correlation_df.iloc[index]
    ref_index1 = data["ref_index1"]
    ref_index2 = data["ref_index2"]
    if ref_index1 in pos and ref_index2 in pos:
        print(ref_index1, ref_index2, exp_comparison_pvalues.iloc[index], np.max([exp_comparison_pvalues.iloc[index], exp_pvalues.iloc[index]]))
        data1.append(np.max([exp_comparison_pvalues.iloc[index], exp_pvalues.iloc[index]]))
print(combine_pvalues(data1))

pos = [817, 908, 2619, 2724]
for index in range(len(correlation_df)):
    data = correlation_df.iloc[index]
    ref_index1 = data["ref_index1"]
    ref_index2 = data["ref_index2"]
    if ref_index1 in pos and ref_index2 in pos:
        print(ref_index1, ref_index2, exp_comparison_pvalues.iloc[index], np.max([exp_comparison_pvalues.iloc[index], exp_pvalues.iloc[index]]))


1437 1449 1.8223342536602726e-15 0.00011995259021815532
1437 1450 5.288312406616294e-13 0.00020148981673063972
1449 1450 8.292598903628237e-55 1.0067891856334518e-15
(104.14039168389853, 3.427889638139926e-20)
817 908 1.3222268936806936e-15 1.3222268936806936e-15
817 2619 0.6274594132991169 0.6274594132991169
817 2724 0.9757682021875592 0.9757682021875592
908 2619 0.9998578863250832 0.9998578863250832
908 2724 0.9499754691365748 0.9499754691365748
2619 2724 0.012045911961679428 0.012045911961679428
